In [9]:
# 자동화하기
# 이어서 쓸 때 save_to_file과 append_to_file 함수에 mode의 파라미터를 'a'로 변경해보기
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
from selenium.webdriver.chrome.options import Options
import pandas as pd
import csv

In [53]:
# 변수 설정
QUERY = '쿠팡'
search_QUERY = urlencode({'subquery': QUERY}, encoding='utf-8')
URL = f"https://www.fmkorea.com/search.php?mid=football_korean&search_keyword={QUERY}&search_target=title_content"

def get_posts(page_num):
    global QUERY
    posts = []
    for page in range(page_num + 1):
        # 페이지 넘기기
        board_link = f"https://www.fmkorea.com/search.php?mid=football_korean&search_keyword={QUERY}&search_target=title_content&page={page+1}"
        req = requests.get(board_link)
        print(req.status_code)
        soup = BeautifulSoup(req.text, 'html.parser')
        # 전체 20개의 게시글의 링크를 posts에 append하는 과정
        tds = soup.find_all('td', {'class': 'title hotdeal_var8'})
        for td in tds:
            post = td.find('a', {'class': 'hx'})
            if post is not None:
                posts.append(post['href'])
    print(f"총 {len(posts)}개의 글 링크를 찾았습니다.")
    # 게시글 링크 csv로 저장
    post_file = open(f"FMKOREA_{QUERY}_{page_num}pages_inner_links.csv", mode='w', encoding='utf-8')
    writer = csv.writer(post_file)
    for post in posts:
        writer.writerow([post])
    post_file.close()
    return posts

def extract_info(url):
    full_url = 'http://fmkorea.com'+url
    chrome_options = Options()
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(full_url)
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    title = soup.find('span', {'class': 'np_18px_span'}).get_text(strip=True)
    user_id = soup.find('a', {'class': lambda x: x and x.startswith('member_')}).get_text(strip=True)
    post_time = soup.find('span', {'class': 'date m_no'}).get_text(strip=True)
    post = soup.find('div', class_='xe_content').get_text(separator = ' ', strip=True)
    # 댓글 수 추출
    try:
        # 댓글 수 추출
        reply_cnt_div = soup.find('div', class_='fdb_tag bg_f_f9 css3pie')
        if reply_cnt_div:
            reply_cnt = int(reply_cnt_div.find('b').text)
        else:
            reply_cnt = 0
    except (AttributeError, ValueError):
        reply_cnt = 0
    reply_content = []
    if reply_cnt != 0:
        #페이지 숫자 확인
        paging_div = soup.find('div', class_ = 'bd_pg clear')
        if paging_div:
            for tag in paging_div.find('strong'):
                if tag.name == 'a' and 'direction' in tag.get('class',[]):
                    continue
                try:
                    cmt_page_max = int(tag.get_text())
                except ValueError:
                    continue
        else:
            cmt_page_max = 1
        # 댓글 정보 추가 (댓글 크롤링 부분에 추가)
        for i in range(cmt_page_max):
            j = cmt_page_max - i
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            # 모든 댓글 항목 찾기
            # 댓글이 포함된 ul 태그 찾기
            comments_ul = soup.find('ul', class_='fdb_lst_ul')

            # 각 댓글 li 태그 찾기
            comments = comments_ul.find_all('li', class_='fdb_itm')
            for comment in comments:
                cmt_reply = False
                reply_prefix = ''
                # 대댓글 여부 확인
                if 're' in comment.get('class', []):
                    cmt_reply = True
                    reply_prefix = 'ㄴ'
                
                # 댓글 작성자 추출
                author = comment.find('a', class_='member_plate').text.strip()
                
                # 댓글 작성 시간 추출
                date = comment.find('span', class_='date').text.strip()
                
                # 댓글 내용 추출
                content = comment.find('div', class_='xe_content').text.strip()
                if cmt_reply:
                    content = f"{reply_prefix} {content}"
                reply_content.append({'author':author, 'text':content, 'date':date})
            if j>1:
                driver.get(f'https://www.fmkorea.com/search.php?mid=football_korean&sort_index=pop&order_type=desc&search_target=title_content&document_srl=7052178769&search_keyword={QUERY}&listStyle=webzine&cpage={j-1}')
                time.sleep(2)

    data = {'title': title, 'user_id': user_id, 'post_time': post_time, 'post': post, 'reply_cnt': reply_cnt, 'reply_content': reply_content}

    # 댓글이 있다면 딕셔너리에 추가
    if reply_cnt != 0:
        for i, reply_info in enumerate(reply_content):
            data[f'reply_{i+1}_author'] = reply_info['author']
            data[f'reply_{i+1}_text'] = reply_info['text']
            data[f'reply_{i+1}_date'] = reply_info['date']
            

    print(url, '완료')
    return data

def get_contents(posts):
    for post_link in posts:
        content = extract_info(post_link)
        append_to_file(f"FMKOREA_{QUERY}_.csv", content)
    return print("모든 작업이 완료되었습니다.")

def save_to_file():
    file = open(f'FMKOREA_{QUERY}_.csv', mode='a', encoding='utf-8')
    writer = csv.writer(file)
    writer.writerow(['title', 'user_id', 'post_time', 'post', 'reply_cnt', 'reply_author', 'reply_content'])
    file.close()
    return file

def append_to_file(file_name, data):
    file = open(file_name, mode='a', encoding='utf-8', newline='')  # newline 추가
    writer = csv.writer(file)

    # Write main post data to CSV
    main_post_row = [
        data['title'],
        data['user_id'],
        data['post_time'],
        data['post'],
        data['reply_cnt']
    ]

    # Check if there are replies
    if 'reply_content' in data and data['reply_cnt'] != 0:
        # Write main post data along with reply data to CSV
        for i, reply_info in enumerate(data['reply_content']):
            writer.writerow(main_post_row + [
                reply_info.get('author', ''),  # Reply author
                reply_info.get('text', ''),  # Reply text
                reply_info.get('date', '')
            ])
    else:
        # Write only main post data to CSV
        writer.writerow(main_post_row + ['']*3)

    file.close()
    return

In [54]:
# 크롤링할 페이지 수 설정
PAGES = 499
# 게시글 링크 수집
post_links = get_posts(PAGES)
save_to_file()
get_contents(post_links)

200
200
총 40개의 글 링크를 찾았습니다.


In [52]:
df = pd.read_csv('FMKOREA_쿠팡_.csv', names=['title','user_id','post_time','post','reply_cnt','reply_author','reply_content','reply_date'])
df=df.drop(0)
df

,title,user_id,post_time,post,reply_cnt,reply_author,reply_content,reply_date
1,TV로보는데 화질 너무 좋다,호세피렐라,2023.07.27 20:04,쿠팡 고마워..,5,춥지않던겨울,티비 몇번에서 하나요,2023.07.27 20:04
2,TV로보는데 화질 너무 좋다,호세피렐라,2023.07.27 20:04,쿠팡 고마워..,5,호세피렐라,ㄴ 춥지않던겨울 쿠팡플레이로 튼거에요,2023.07.27 20:04
3,TV로보는데 화질 너무 좋다,호세피렐라,2023.07.27 20:04,쿠팡 고마워..,5,간지붐송,ㄴ 춥지않던겨울 쿠플 독점이라 TV에 연결해서 보는 사람들 꽤 있음,2023.07.27 20:05
4,TV로보는데 화질 너무 좋다,호세피렐라,2023.07.27 20:04,쿠팡 고마워..,5,전진우,현장음도 적절하니 넘 좋음ㅋㅋㅋ,2023.07.27 20:04
5,TV로보는데 화질 너무 좋다,호세피렐라,2023.07.27 20:04,쿠팡 고마워..,5,Nasir,너무 좋다 화질,2023.07.27 20:05
...,...,...,...,...,...,...,...,...
2864,7/16 K리그 라인업,킹짱구,2023.07.15 23:52,"[ 전북 vs 수 원 F C ] 라인업 - 19:00 킥오프 | 쿠팡 플레이 , J...",8,압델하크누리,ㄴ 핀란드망고 한숨쉬며 보던 개리그를한숨놓고 보겠네,2023.07.15 23:52
2865,7/16 K리그 라인업,킹짱구,2023.07.15 23:52,"[ 전북 vs 수 원 F C ] 라인업 - 19:00 킥오프 | 쿠팡 플레이 , J...",8,야스,지나치게 열심히 하는 판쟝이 들어왔다,2023.07.15 23:52
2866,7/16 K리그 라인업,킹짱구,2023.07.15 23:52,"[ 전북 vs 수 원 F C ] 라인업 - 19:00 킥오프 | 쿠팡 플레이 , J...",8,엔데버,스카이 스포츠 분신술씀?,2023.07.15 23:55
2867,7/16 K리그 라인업,킹짱구,2023.07.15 23:52,"[ 전북 vs 수 원 F C ] 라인업 - 19:00 킥오프 | 쿠팡 플레이 , J...",8,킹짱구,ㄴ 엔데버 며칠전에 만들어 논거라 바꼈음 ㅎㅅㅎ,2023.07.16 00:07
